In [ ]:
import os

import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

import IPython.display as ipd
from tqdm import tqdm
import numpy as np
import librosa
import scipy
import uuid
import pandas as pd
from pathlib import Path
import datasets
from datasets import load_dataset
from huggingface_hub import login
import io
import yaml

from openspeechtointent.model import CitrinetModel
mdl = CitrinetModel()


# Load Config for the model

In [ ]:

# Load the YAML config file

with open('../wakeword_training_configs/angela.yml', 'r') as file:
    config = yaml.safe_load(file)

# Load XTTSv2 model

In [ ]:
# Load the model

config = XttsConfig()
config.load_json("/home/dscripka/dscripkaDrive_nvme_fast/models/XTTS-v2/config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/home/dscripka/dscripkaDrive_nvme_fast/models/XTTS-v2", eval=True)
_ = model.cuda()

# Test Generation

In [ ]:
# List all speakers
model.speaker_manager.speakers.keys()

In [ ]:
# Test single generation
speaker = model.speaker_manager.speakers['Gracie Wise']
emb = speaker['speaker_embedding']
cond_latent = speaker['gpt_cond_latent']

# speaker_nm = "Tammie Ema"
out = model.inference(
    "Angela",
    "en",  # changing this to other languages sometimes results in other accents, but also sometimes degrades the pronunciation
    cond_latent,
    emb,
    temperature=0.5,
    speed=1.0
)

ipd.Audio(out['wav'], rate=24000)

# Use XTTSv2 to Clone Commmon Voice 17 Speakers and Generate Audio

In [ ]:
# Login to HF Hub
login(token="")  # Add your access token here

# Load the dataset with streaming enabled and shuffling
dataset = load_dataset("mozilla-foundation/common_voice_17_0", 'en', streaming=True, split="validation", trust_remote_code=True)

In [ ]:
# Clone common voice speakers and generate synthetic training data

text = "Angela"
output_dir = "<your output directory here>"

N = 1000
cnt = 0
for i, example in tqdm(enumerate(iter(dataset)), total=1000):
    # Access the audio from common voice
    audio_array = example['audio']['array']
    sampling_rate = example['audio']['sampling_rate']

    # Make an audio file-like object
    audio_fp = io.BytesIO()
    scipy.io.wavfile.write(audio_fp, sampling_rate, audio_array)
    audio_fp.seek(0)

    # Get the conditioning latents and speaker embeddings
    cond_latent, emb = model.get_conditioning_latents(audio_path = audio_fp)
    
    # Generate the audio
    out = model.inference(
        text.strip("?") if np.random.rand() > 0.5 else text,
        "en",  # changing this to other languages sometimes results in other accents, but also sometimes degrades the pronunciation
        cond_latent,
        emb,
        temperature=0.5,
        speed=1.0
    )

    # Resample to 16khz with librosa
    resampled = librosa.resample(out['wav'], orig_sr=24000, target_sr=16000)

    # Keep only the first 1.3 seconds
    resampled = resampled[0:int(16000*1.3)]

    # Convert to int16
    resampled = (resampled * 32767).astype(np.int16)

    # Save to file
    scipy.io.wavfile.write(os.path.join(output_dir, f"{uuid.uuid4().hex}.wav"), 16000, resampled)

    if cnt == N:
        break

In [ ]:
# Listen to the generated samples

generated_clips = [str(i) for i in Path(output_dir).glob("*.wav")]

print(len(generated_clips))

ipd.Audio(generated_clips[14])

In [ ]:
# Validate the clips using openSpeechToIntent

generated_clips = [str(i) for i in Path(output_dir).glob("*.wav")]
print(len(generated_clips))

intent = config["target_phrase"]

all_scores = []
for i in tqdm(generated_clips):
    sr, dat = scipy.io.wavfile.read(i)
    matched_intents, scores, durations = mdl.match_intents(dat, intent, softmax_scores=False)
    all_scores.append(scores.max())

# Save the results to a csv file
pd.DataFrame({"file_path": generated_clips, "scores": all_scores}).to_csv("../data/positive_training_clips_xttsv2_cv_clone_quality_scores.csv", index=False)

